# Micu Sebastian - 1240F - Devoir 1

Les librairies que j'ai utilisées

In [158]:
from collections import defaultdict
import random
import math
import numpy as np

La class *Board*

Pour cette classe, la valeur d'utilité peut être:
  

*   **-1** si le joueur qui cherche à obtenir un minimum va gagner;
*   **0** s'il y a égalité;
*   **1** si le joueur qui cherche à obtenir un maximum va gagner.

In [159]:
class Board(defaultdict):
    empty = '.'
    used = '#'
    
    def __init__(self, width=8, height=8, current_player=None, **kwds):
        self.__dict__.update(width=width, height=height, current_player=current_player, **kwds)
 
    def __missing__(self, pos):
        a, b = pos
        if (a < self.width and a >= 0) and (b < self.height and b >= 0):
          return self.empty
        else:
          return self.used
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.current_player)
    
    def __repr__(self):
        def row(y): 
            return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'
    
    def update_board(self, changes: dict, **kwds) -> 'Board':
        board = Board(self.width, self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

La fonction *k_in_row* pour calculer les symboles 'X' et 'O' dans en ligne.

In [160]:
def k_in_row(board, player, square, k):
    def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))

La class *TicTacToe*

In [161]:
class TicTacToe:
    
    def __init__(self, height=3, width=3, k=3):
        self.k = k
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(width, height, 'X', utility=0) # Le Board avec les paramètres de largeur et de hauteur, où X joue en premier et la valeur d'utilité est 0.
 
    def actions(self, board):
        return self.squares - set(board) # Définissez les déplacements possibles pour les positions autorisées.

    def utility(self, board, player):
        return board.utility if player == 'X' else -board.utility

    def make_move(self, board, square):
        player = board.current_player
        board = board.update_board({square: player}, current_player=('O' if player == 'X' else 'X')) # Mettez à jour le Board au cas où le joueur actuel placerait son symbole
        res = k_in_row(board, player, square, self.k)
        board.utility = (0 if not res else +1 if player == 'X' else -1) # Mettez à jour l'utility du Board avec le symbole correspondant pour chaque joueur.

        return board
    
    def end(self, board):
        return board.utility != 0 or len(self.squares) == len(board) # Vérifie si le jeu est terminé.
 
    def draw_board(self, board):
        print(board)

Robot aléatoire

In [162]:
def random_player(game, state):
    return random.choice(list(game.actions(state))) # Définition d'un joueur qui utilise toujours des mouvements aléatoires.

Joueur d'algorithmes

In [176]:
def player_old(search_algorithm):
    return lambda game, state: search_algorithm(game, state)[1] # Définition d'un joueur généraliste qui utilise une stratégie (algorithme de recherche).

Joueur d'algorithmes pour le *Monte_Carlo_Tree_Search*

In [178]:
def player(search_algorithm):
    return lambda game, state: search_algorithm(game, state)

La fonction play_game qui reçoit le jeu actuel à jouer et une stratégie.

In [164]:
def play_game(game, strategies: dict):
    state = game.initial
    while not game.end(state):
      p = state.current_player
      m = strategies[p](game, state)
      state = game.make_move(state, m)
      print ("Player {} move: {}" .format(p, m))
      print (state)
    
    return state

 L'algorithme Minimax

In [165]:
infinity = math.inf

def minimax_search(game, state):
    player = state.current_player

    def max_value(state):
        if game.end(state):
          return game.utility(state, player), None
        
        value, move = -infinity, None

        for a in game.actions(state):
          index, non = min_value(game.make_move(state, a))
          if index > value:
            value, move = index, a

        return value, move

    def min_value(state):
        if game.end(state):
          return game.utility(state, player), None
        
        value, move = +infinity, None

        for a in game.actions(state):
          index, non = max_value(game.make_move(state, a))
          if index < value:
            value, move = index, a
        
        return value, move

    return max_value(state)

Démarrer et tester le jeu avec un random_player et un minimax_search player.

In [177]:
play_game(TicTacToe(), dict(X=random_player, O=player_old(minimax_search))).utility

Player X move: (1, 1)
. . .
. X .
. . .

Player O move: (0, 0)
O . .
. X .
. . .

Player X move: (2, 1)
O . .
. X X
. . .

Player O move: (0, 1)
O . .
O X X
. . .

Player X move: (1, 2)
O . .
O X X
. X .

Player O move: (1, 0)
O O .
O X X
. X .

Player X move: (0, 2)
O O .
O X X
X X .

Player O move: (2, 0)
O O O
O X X
X X .



-1

Tache #1 : ConnectFour()

In [167]:
class ConnectFour(TicTacToe):
    
    def __init__(self): super().__init__(width=7, height=6, k=4) # J'ai appelé la superclasse et mis la largeur à 7, la hauteur à 6 et la combinaison gagnante à 4.

    def actions(self, board):
        return {(x, y) for (x, y) in self.squares - set(board) if y == board.height - 1 or (x, y + 1) in board} # La fonction qui oblige le joueur à n'opérer que dans la cellule vide la plus basse d'une colonne.

Démarrer et tester le jeu avec deux random_player.

In [170]:
play_game(ConnectFour(), dict(X=random_player, O=random_player)).utility

Player X move: (2, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . X . . . .

Player O move: (1, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. O X . . . .

Player X move: (0, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
X O X . . . .

Player O move: (2, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X O X . . . .

Player X move: (6, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X O X . . . X

Player O move: (2, 3)
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
. . O . . . .
X O X . . . X

Player X move: (0, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X . O . . . .
X O X . . . X

Player O move: (6, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X . O . . . O
X O X . . . X

Player X move: (3, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
X . O . . . O
X O X X . . X

Player O move: (6, 3)
. . . . . . .
.

-1

Tache #2 : Upper Confidence Bound for Trees

La class *Monte Carlo Tree Node*

In [171]:
class MCTN:
    def __init__(self, parent=None, state=None, U=0, N=0):
        self.__dict__.update(parent=parent, state=state, U=U, N=N) # Le nœud dans l'arbre de recherche Monte Carlo garde une trace des états enfants.
        self.children = {}
        self.actions = None


def ucb(n, C=1.4):
    return np.inf if n.N == 0 else n.U / n.N + C * np.sqrt(np.log(n.parent.N) / n.N) # Trouvez l’action qui maximise cette formule.

In [191]:
def monte_carlo_tree_search(game, state, N=50): # N représente le nombre de simulations avant d'effectuer un mouvement.
    def selection(n): # Une stratégie de sélection d'une action à exploiter (choix d'un nœud feuille dans l'arbre).
        if n.children:
            return selection(max(n.children.keys(), key=ucb)) # Sélectionnez un nœud feuille dans l'arborescence.
        else:
            return n

    def expansion(n): # La construction d'un nouveau nœud dans l'arbre (ajouter les états des enfants au nœud de la feuille)
        if not n.children and not game.end(n.state):
            n.children = {MCTN(state=game.make_move(n.state, action), parent=n): action for action in game.actions(n.state)}
        return selection(n) # Développez le nœud feuille en ajoutant tous ses états d'enfants.

    def simulation(game, state):
        player = state.current_player
        while not game.end(state):
            action = random.choice(list(game.actions(state))) # Simulez l'utilité de l'état actuel en choisissant au hasard une étape.
            state = game.make_move(state, action)
        v = game.utility(state, player)
        return -v

    def backpropagation(n, utility):
        if utility > 0:
            n.U += utility
        n.N += 1
        if n.parent:
            backpropagation(n.parent, -utility) # Renvoyer l'utility à tous les nœuds parents.

    root = MCTN(state=state)

    for _ in range(N): # Après N cas, nous déterminons quel est le meilleur mouvement après avoir traversé les quatre phases.
        leaf = selection(root)
        child = expansion(leaf)
        result = simulation(game, child.state)
        backpropagation(child, result)

    max_state = max(root.children, key=lambda p: p.N) # Évaluez le statut final et déterminez si une récompense est calculée.

    return root.children.get(max_state)

Démarrer et tester le jeu avec un random_player et un monte_carlo_tree_search player.

In [193]:
play_game(ConnectFour(), dict(X=random_player, O=player(monte_carlo_tree_search))).utility

Player X move: (5, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X .

Player O move: (6, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X O

Player X move: (1, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. X . . . X O

Player O move: (0, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
O X . . . X O

Player X move: (5, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X .
O X . . . X O

Player O move: (4, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X .
O X . . O X O

Player X move: (6, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X X
O X . . O X O

Player O move: (3, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . X X
O X . O O X O

Player X move: (4, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . X X X
O X . O O X O

Player O move: (3, 4)
. . . . . . .
.

-1

Tache #3 : Query player

In [194]:
def query_player(game, state):
    print("Usable moves: {}" .format(game.actions(state))) # Afficher les mouvements disponibles / légaux.
    print("")
    move = None
    if game.actions(state):
        move_string = input('Choose your move: ') # Faire un mouvement en interrogeant l'entrée.
        try:
            move = eval(move_string) # Analyse de l'expression passée à cette méthode et exécute le code.
        except NameError:
            move = move_string
    else:
        print('Unavailable moves: Loading next player')
    return move

Démarrer et tester le jeu avec un random_player et un query_player.

In [196]:
play_game(ConnectFour(), dict(X=random_player, O=query_player)).utility

Player X move: (3, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . X . . .

Usable moves: {(5, 5), (4, 5), (1, 5), (0, 5), (2, 5), (3, 4), (6, 5)}

Choose your move: 3,4
Player O move: (3, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
. . . X . . .

Player X move: (0, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
X . . X . . .

Usable moves: {(5, 5), (3, 3), (4, 5), (1, 5), (0, 4), (2, 5), (6, 5)}

Choose your move: 2,5
Player O move: (2, 5)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . . O . . .
X . O X . . .

Player X move: (2, 4)
. . . . . . .
. . . . . . .
. . . . . . .
. . . . . . .
. . X O . . .
X . O X . . .

Usable moves: {(5, 5), (3, 3), (4, 5), (1, 5), (2, 3), (0, 4), (6, 5)}

Choose your move: 2,3
Player O move: (2, 3)
. . . . . . .
. . . . . . .
. . . . . . .
. . O . . . .
. . X O . . .
X . O X . . .

Player X move: (3, 3)
. . . . . . .
. . . . . . .
. . . . . . .
. . O X . . 

-1